<a href="https://colab.research.google.com/github/Jesimsim/ds-section2-project/blob/main/section2_project_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **아파트 실거래가 예측**

In [2]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import math

# from sklearn. import
# from sklearn.metric import f1_score

#### **필요코드 설치**

In [1]:
# Google Colab을 사용하는 경우 해당 셀을 실행하세요
import sys

if "google.colab" in sys.modules:
    # Install packages in Colab
    !pip install category_encoders==2.*
    !pip install eli5
    !pip install pandas-profiling==2.*
    !pip install pdpbox
    !pip install --upgrade xgboost


import xgboost
import warnings

warnings.filterwarnings("ignore")
# xgboost.config.set_config(verbosity=0)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 324 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 4.2 MB/s 
     |████████████████████████████████| 133 kB 5.2 MB/s 
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107748 sha256=4977909e18b06984d1249578199e958f226f1f099b4662151a88f9e5e6689363
  Stored in directory: /root/.cache/pip/wheels/cc/3c/96/3ead31a8e6c20fc0f1a707fde2e05d49a80b1b4b30096573be
Successfully built eli5
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
notebook 5.7.16 req

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 255.9 MB 43 kB/s 
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


#### **데이터 불러오기 / 병합하기 - 아파트데이터**

In [3]:
# 데이터 불러오기 및 병합

# 데이터 확인
import os
os.listdir('/content/AptALL')

FileNotFoundError: ignored

In [44]:
forders= os.listdir('/content/AptALL')
%cd /content/AptALL

/content/AptALL


In [43]:
# 데이터 병합 - 월별
df = pd.DataFrame()

for files in forders:
    df_1 = pd.read_csv(files, encoding='cp949', encoding_errors='ignore',  skiprows = [i for i in range(0, 15) ])
    df = pd.concat([df, df_1], axis=0)

df.to_csv("Apt_2022_all.csv", encoding='utf-8')

In [ ]:
# 병합오류 확인
pd.read_csv('Apt_2017_all.csv', encoding='utf-8', encoding_errors='ignore') #2022(다시병합), 2017(언어 오류, 병합오류). 2013(병합오류)

In [40]:
#오류데이터프레임 수정 > 업로드 제대로 마칠때까지 기다리기
df = pd.DataFrame()

for files in forders:
    df_1 = pd.read_csv(files, encoding='cp949',  skiprows = [i for i in range(0, 15) ])
    df = pd.concat([df, df_1], axis=0)

df = df.drop(['Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18','Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22'], axis=1)
df.columns
df.to_csv("Apt_2013_all.csv", encoding='utf-8')

In [9]:
# 데이터 병합 - 총

import os

forders= os.listdir('/content/AptALL')
%cd /content/AptALL

df = pd.DataFrame()

for files in forders:
    df_1 = pd.read_csv(files, encoding='utf-8', encoding_errors='ignore', index_col=0)
    df = pd.concat([df, df_1], axis=0)


df.to_csv("Apt_total.csv", encoding='utf-8')

/content/AptALL


IsADirectoryError: ignored

In [77]:
# 에러파일 확인
forders= os.listdir('/content/AptALL')
%cd /content/AptALL
pd.read_csv('Apt_2013_all.csv', encoding='utf-8', encoding_errors='ignore') #2013 unnamed:칼럼 2개


/content/AptALL


,Unnamed: 0,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,거래유형,중개사소재지
0,0,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,104.43,202102,9,"145,000",6,1984,개포로 307,20210223.0,-,-
1,1,서울특별시 강남구 개포동,185,185.0,0.0,개포주공6단지,53.06,202102,1,"200,000",12,1983,개포로 516,NaN,-,-
2,2,서울특별시 강남구 개포동,185,185.0,0.0,개포주공6단지,73.02,202102,28,"240,500",7,1983,개포로 516,NaN,-,-
3,3,서울특별시 강남구 개포동,649,649.0,0.0,경남2차,182.20,202102,5,"327,000",11,1984,언주로 110,NaN,-,-
4,4,서울특별시 강남구 개포동,649,649.0,0.0,경남2차,186.16,202102,6,"332,000",14,1984,언주로 110,NaN,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43413,1438,서울특별시 중랑구 신내동,650,650.0,0.0,신내6대주,59.76,202111,30,"75,000",12,1996,신내로19길 42,NaN,중개거래,서울 중랑구
43414,1439,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,59.49,202111,5,"80,000",20,2014,신내역로1길 85,NaN,중개거래,"서울 동대문구, 서울 중랑구"
43415,1440,서울특별시 중랑구 신내동,654,654.0,0.0,진로아파트,101.86,202111,4,"100,000",7,1995,신내로17길 41,NaN,중개거래,서울 중랑구
43416,1441,서울특별시 중랑구 신내동,656,656.0,0.0,화성.두산아파트,84.99,202111,22,"86,000",14,1995,신내로 155,NaN,중개거래,서울 중랑구


In [81]:
# 에러파일 수정
df = pd.read_csv('Apt_2013_all.csv', encoding='utf-8', encoding_errors='ignore')
df = df.drop('Unnamed: 0.1', axis=1)
df.to_csv("Apt_2013_all(2).csv", encoding='utf-8')

In [12]:
# 에러파일 확인 2 / 언어오류
df = pd.DataFrame()
forder = [
  'Apt_2019_all.csv',
 'Apt_2013_all.csv',
 'Apt_2017_all.csv',
 'Apt_2010_all.csv',
 'Apt_2011_all.csv',
 'Apt_2022_all.csv',
 'Apt_2015_all.csv',
 'Apt_2012_all.csv',
 'Apt_2020_all.csv',
 'Apt_2018_all.csv',
 'Apt_2021_all.csv',
 'Apt_2014_all.csv',
 'Apt_2016_all.csv']


for files in forders:
    df_1 = pd.read_csv(files, encoding='utf-8', encoding_errors='ignore')
    df = pd.concat([df, df_1], axis=0)

df.head()
# df.to_csv("Apt_check1.csv", encoding='utf-8')

,Unnamed: 0,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,거래유형,중개사소재지,Unnamed: 0.1
0,0,서울특별시 강남구 개포동,12-2,12.0,2.0,개포자이(12-2),168.42,201912,11,"202,000",5.0,2004.0,개포로109길 69,NaN,-,-,NaN
1,1,서울특별시 강남구 개포동,12-2,12.0,2.0,개포자이(12-2),170.32,201912,11,"218,000",9.0,2004.0,개포로109길 69,NaN,-,-,NaN
2,2,서울특별시 강남구 개포동,141,141.0,0.0,개포주공1단지,41.98,201912,3,"204,000",5.0,1982.0,개포로 310,NaN,-,-,NaN
3,3,서울특별시 강남구 개포동,141,141.0,0.0,개포주공1단지,50.38,201912,4,"220,000",2.0,1982.0,개포로 310,NaN,-,-,NaN
4,4,서울특별시 강남구 개포동,141,141.0,0.0,개포주공1단지,41.98,201912,5,"205,000",1.0,1982.0,개포로 310,NaN,-,-,NaN


In [6]:
# 오류 확인
pd.read_csv('Apt_total.csv', encoding='utf-8', encoding_errors='ignore')

,index,Unnamed: 0,Unnamed: 0.1,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,거래유형,중개사소재지,Unnamed: 0.1.1
0,0,0,0,서울특별시 강남구 개포동,12-2,12.0,2.0,개포자이(12-2),168.42,201912,11,"202,000",5.0,2004.0,개포로109길 69,NaN,-,-,NaN
1,1,1,1,서울특별시 강남구 개포동,12-2,12.0,2.0,개포자이(12-2),170.32,201912,11,"218,000",9.0,2004.0,개포로109길 69,NaN,-,-,NaN
2,2,2,2,서울특별시 강남구 개포동,141,141.0,0.0,개포주공1단지,41.98,201912,3,"204,000",5.0,1982.0,개포로 310,NaN,-,-,NaN
3,3,3,3,서울특별시 강남구 개포동,141,141.0,0.0,개포주공1단지,50.38,201912,4,"220,000",2.0,1982.0,개포로 310,NaN,-,-,NaN
4,4,4,4,서울특별시 강남구 개포동,141,141.0,0.0,개포주공1단지,41.98,201912,5,"205,000",1.0,1982.0,개포로 310,NaN,-,-,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924455,924455,110317,10477,서울특별시 중랑구 중화동,450,450.0,0.0,한신아파트(103~109),59.76,201604,21,"30,000",7.0,1997.0,동일로 752,NaN,-,-,NaN
924456,924456,110318,10478,서울특별시 중랑구 중화동,450,450.0,0.0,한신아파트(103~109),84.03,201604,21,"39,000",15.0,1997.0,동일로 752,NaN,-,-,NaN
924457,924457,110319,10479,서울특별시 중랑구 중화동,450,450.0,0.0,한신아파트(103~109),59.76,201604,25,"32,000",21.0,1997.0,동일로 752,NaN,-,-,NaN
924458,924458,110320,10480,서울특별시 중랑구 중화동,450,450.0,0.0,한신아파트(103~109),50.37,201604,26,"27,200",6.0,1997.0,동일로 752,NaN,-,-,NaN


- 파일 가져올때 https://hyunsitstory.tistory.com/entry/python-Jupyter-폴더-내-여러-개의-csv파일을-하나의-데이터-프레임으로-한-번에-합치기
- https://boleumdal.tistory.com/entry/python-폴더-안에-있는-파일-한번에-불러오기 
- 액셀 열때, 텍스트가져오기 마법사 https://support.microsoft.com/ko-kr/office/%ED%85%8D%EC%8A%A4%ED%8A%B8-%EA%B0%80%EC%A0%B8%EC%98%A4%EA%B8%B0-%EB%A7%88%EB%B2%95%EC%82%AC-c5b02af6-fda1-4440-899f-f78bafe41857


#### **데이터불러오기**

In [21]:
df = pd.read_csv('Apt_total.csv', encoding='utf-8', encoding_errors='ignore')
df = pd.DataFrame(df, columns=['시군구','번지','본번','부번','단지명','전용면적','계약년월'
    ,'계약일','거래금액(만원)','층','건축년도','도로명','해제사유발생일','거래유형','중개사소재지'])


In [14]:
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)


In [29]:
df.head()

,시군구,번지,본번,부번,단지명,전용면적,계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,거래유형,중개사소재지
0,서울특별시 강남구 개포동,12-2,12.0,2.0,개포자이(12-2),NaN,201912.0,11.0,"202,000",5.0,2004.0,개포로109길 69,NaN,-,-
1,서울특별시 강남구 개포동,12-2,12.0,2.0,개포자이(12-2),NaN,201912.0,11.0,"218,000",9.0,2004.0,개포로109길 69,NaN,-,-
2,서울특별시 강남구 개포동,141,141.0,0.0,개포주공1단지,NaN,201912.0,3.0,"204,000",5.0,1982.0,개포로 310,NaN,-,-
3,서울특별시 강남구 개포동,141,141.0,0.0,개포주공1단지,NaN,201912.0,4.0,"220,000",2.0,1982.0,개포로 310,NaN,-,-
4,서울특별시 강남구 개포동,141,141.0,0.0,개포주공1단지,NaN,201912.0,5.0,"205,000",1.0,1982.0,개포로 310,NaN,-,-


In [32]:
df_new = pd.read_csv('Apt_2016  _all.csv', encoding='utf-8', encoding_errors='ignore')
df_new.head()

,Unnamed: 0,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,거래유형,중개사소재지
0,0,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,104.43,201607,10,"104,500",14,1984,개포로 307,NaN,-,-
1,1,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,161.00,201607,15,"156,000",10,1984,개포로 307,NaN,-,-
2,2,서울특별시 강남구 개포동,12-2,12.0,2.0,개포자이(12-2),153.80,201607,9,"150,000",21,2004,개포로109길 69,NaN,-,-
3,3,서울특별시 강남구 개포동,12-2,12.0,2.0,개포자이(12-2),134.30,201607,14,"128,000",13,2004,개포로109길 69,NaN,-,-
4,4,서울특별시 강남구 개포동,12-2,12.0,2.0,개포자이(12-2),134.30,201607,18,"124,500",10,2004,개포로109길 69,NaN,-,-
